In [1]:
!pip install cupy-cuda12x

In [4]:
import os
import pandas as pd
import h5py
import numpy as np

# === CONFIGURATION ===
data_folder = "/kaggle/input/stanford-earthquake-dataset-stead"   # Path to folder containing .csv and .hdf5 files
output_folder = "/kaggle/working/npz_output"  # Where to save the .npz files
os.makedirs(output_folder, exist_ok=True)

# === LOOP THROUGH FILES ===
for csv_file in os.listdir(data_folder):
    if not csv_file.endswith(".csv"):
        continue

    base_name = os.path.splitext(csv_file)[0]
    hdf5_file = os.path.join(data_folder, f"{base_name}.hdf5")
    csv_path = os.path.join(data_folder, csv_file)

    if not os.path.exists(hdf5_file):
        print(f"⚠️ Skipping {csv_file}: matching HDF5 file not found.")
        continue

    print(f"\n📂 Processing: {csv_file} and {hdf5_file}")

    # --- Step 1: Read CSV ---
    df = pd.read_csv(csv_path)
    print(f"Total events in CSV: {len(df)}")

    # --- Step 2: Filter Events ---
    df = df[
        (df.trace_category == 'earthquake_local') &
        (df.source_distance_km <= 20) &
        (df.source_magnitude > 4)
    ]
    print(f"Selected events: {len(df)}")

    if len(df) == 0:
        continue

    # --- Step 3: Open HDF5 File ---
    with h5py.File(hdf5_file, 'r') as dtfl:
        for idx, trace_name in enumerate(df['trace_name']):
            if f"data/{trace_name}" not in dtfl:
                print(f"⚠️ Trace {trace_name} not found in {hdf5_file}. Skipping.")
                continue

            dataset = dtfl[f"data/{trace_name}"]
            data = np.array(dataset)  # Shape: (N, 3)
            
            # --- Step 4: Extract attributes ---
            attrs = {k: dataset.attrs[k] for k in dataset.attrs.keys()}

            # --- Step 5: Save waveform and metadata ---
            save_name = f"{base_name}_{trace_name}.npz"
            save_path = os.path.join(output_folder, save_name)

            np.savez_compressed(save_path, 
                                waveform=data, 
                                attributes=attrs)

            print(f"✅ Saved {save_name} (shape={data.shape})")

print("\n🎉 All files processed and saved to:", output_folder)



📂 Processing: merge.csv and /kaggle/input/stanford-earthquake-dataset-stead/merge.hdf5


/tmp/ipykernel_37/689605949.py:27: DtypeWarning: Columns (7,11,13,14,24,25,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Total events in CSV: 1265657
Selected events: 352
✅ Saved merge_A64.CN_20130711201601_EV.npz (shape=(6000, 3))
✅ Saved merge_AC04.C1_20140616231955_EV.npz (shape=(6000, 3))
✅ Saved merge_AC04.C1_20140720063145_EV.npz (shape=(6000, 3))
✅ Saved merge_AC04.C1_20140720063148_EV.npz (shape=(6000, 3))
✅ Saved merge_AKRB.AV_20100820164042_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090406022743_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090406163802_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090406231531_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090407174734_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090407213423_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090408225648_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090409005253_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090409031446_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090409043241_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_20090409193816_EV.npz (shape=(6000, 3))
✅ Saved merge_AQU.MN_2009041321

In [ ]:
"""
Upload filtered seismic waveform data (.npz) to Edge Impulse using Python SDK.

Each .npz file must contain:
  - waveform: ndarray of shape (samples, 3)
  - attributes: metadata dict (optional)

This script:
  - Reads earthquake and noise waveform .npz files
  - Builds a consistent data array and label list
  - Uploads to Edge Impulse via upload_numpy()

Author: Nithin Aravindan (Edge AI Seismic Project)
"""

import os
import numpy as np
import edgeimpulse as ei

# === CONFIGURATION ===
EI_API_KEY = "ei_eb03aa1fa9592e163b72614732eecc5756bc5250c1778d48150ca837df74ca23"   # 🔑 Replace this with your actual API key
earthquake_folder = r"C:/Users/nithi/Downloads/_output_/npz_output"
noise_folder = r"C:/Users/nithi/Downloads/_output_/npz_noise_output"
SAMPLE_RATE_HZ = 100  # Adjust if known (e.g., 100Hz means one sample every 10ms)
CATEGORY = "training"  # or "testing"

# --- Initialize Edge Impulse SDK ---
ei.API_KEY = EI_API_KEY

# === Collect all NPZ files ===
def load_npz_data(folder_path, label):
    data_list = []
    labels = []
    for file in os.listdir(folder_path):
        if not file.endswith(".npz"):
            continue
        file_path = os.path.join(folder_path, file)
        npz = np.load(file_path, allow_pickle=True)
        waveform = npz["waveform"]

        # Optional: pad or trim to fixed length
        # Example: enforce 3000 samples
        TARGET_LENGTH = 3000
        if waveform.shape[0] < TARGET_LENGTH:
            pad_len = TARGET_LENGTH - waveform.shape[0]
            waveform = np.pad(waveform, ((0, pad_len), (0, 0)), mode="constant")
        elif waveform.shape[0] > TARGET_LENGTH:
            waveform = waveform[:TARGET_LENGTH, :]

        data_list.append(waveform)
        labels.append(label)
    return data_list, labels


print("📂 Loading earthquake data...")
earthquake_data, earthquake_labels = load_npz_data(earthquake_folder, "earthquake_local")

print("📂 Loading noise data...")
noise_data, noise_labels = load_npz_data(noise_folder, "noise")

# === Combine and convert to numpy array ===
all_data = np.array(earthquake_data + noise_data)
all_labels = earthquake_labels + noise_labels

print(f"✅ Total samples ready for upload: {len(all_labels)}")
print(f"Data shape: {all_data.shape}")  # (num_samples, num_points, 3)

# === Sensor definition ===
sensors = [
    {"name": "E_channel", "units": "counts"},
    {"name": "N_channel", "units": "counts"},
    {"name": "Z_channel", "units": "counts"},
]

# === Upload to Edge Impulse ===
print("\n🚀 Uploading to Edge Impulse...")
response = ei.experimental.data.upload_numpy(
    data=all_data,
    labels=all_labels,
    sensors=sensors,
    sample_rate_ms=1000 / SAMPLE_RATE_HZ,
    metadata={"project": "seismic_ai", "source": "merge_dataset"},
    category=CATEGORY,
)

# === Upload summary ===
if len(response.fails) == 0:
    print("\n🎉 All uploads successful!")
else:
    print(f"\n⚠️ {len(response.fails)} uploads failed:")
    for fail in response.fails:
        print(f"- {fail}")


In [ ]:
import os
import pandas as pd
import h5py
import numpy as np

# === CONFIGURATION ===
data_folder = "/kaggle/input/stanford-earthquake-dataset-stead"    # Folder containing .csv and .hdf5 files
output_folder = "/kaggle/working/npz_noise_output"  # Output folder for noise data
os.makedirs(output_folder, exist_ok=True)

# === LOOP THROUGH FILES ===
for csv_file in os.listdir(data_folder):
    if not csv_file.endswith(".csv"):
        continue

    base_name = os.path.splitext(csv_file)[0]
    hdf5_file = os.path.join(data_folder, f"{base_name}.hdf5")
    csv_path = os.path.join(data_folder, csv_file)

    if not os.path.exists(hdf5_file):
        print(f"⚠️ Skipping {csv_file}: matching HDF5 file not found.")
        continue

    print(f"\n📂 Processing: {csv_file} and {hdf5_file}")

    # --- Step 1: Read CSV ---
    df = pd.read_csv(csv_path)
    print(f"Total events in CSV: {len(df)}")

    # --- Step 2: Filter for Noise Data ---
    # Typically, trace_category column includes 'noise'
    df = df[df.trace_category == 'noise']
    print(f"Selected noise events: {len(df)}")

    if len(df) == 0:
        continue

    # --- Step 3: Open HDF5 File ---
    with h5py.File(hdf5_file, 'r') as dtfl:
        for idx, trace_name in enumerate(df['trace_name']):
            data_path = f"data/{trace_name}"
            if data_path not in dtfl:
                print(f"⚠️ Trace {trace_name} not found in {hdf5_file}. Skipping.")
                continue

            dataset = dtfl[data_path]
            data = np.array(dataset)  # (samples, 3 channels)

            # --- Step 4: Extract attributes ---
            attrs = {k: dataset.attrs[k] for k in dataset.attrs.keys()}

            # --- Step 5: Save waveform and metadata ---
            save_name = f"{base_name}_{trace_name}_noise.npz"
            save_path = os.path.join(output_folder, save_name)

            np.savez_compressed(save_path,
                                waveform=data,
                                attributes=attrs)

            print(f"✅ Saved {save_name} (shape={data.shape})")

print("\n🎉 All noise waveforms processed and saved to:", output_folder)
